In [1]:
#%matplotlib inline
import os
os.chdir('U:\\GitHubClones\\SALTSeniorDesign')
os.getcwd()
import xlwings as xw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from collections import OrderedDict
import re
from datetime import datetime

In [2]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
wb = xw.Book('Salt.xlsm')
setup = wb.sheets('Setup')
output = wb.sheets('Output')
strats = wb.sheets('Strat Tables')
performance = wb.sheets('Performance')
summ = wb.sheets('Summary')

# Path For the Current deal that is selected based on Setup
directory = setup['Setup.Directory'].value
shelf = setup['Setup.Shelf'].value
shelfList = setup['Shelf_List'].value
deal = setup['Setup.Deal'].value
report = setup['Setup.Report'].value
num_Deals = setup['Setup.Num_Deals'].value
max_Report = setup['Setup.Max_Report_Num'].value
#statsRange = summ['Summary.Statistics'].value

In [4]:
tapes = np.array(setup['Tapes'].value)
tapes = tapes[0:int(max_Report),0:int(num_Deals)]
dfTapes = pd.DataFrame(data=tapes[1:,1:], columns=tapes[0,1:])
dealList = list(dfTapes.columns)

In [5]:
dfTapes.head()

,AMCAR 2017-1,AMCAR 2017-2,AMCAR 2017-3,DRIVE 2017-1,DRIVE 2017-2,DRIVE 2017-3,SDART 2017-1,SDART 2017-2,SDART 2017-3
0,2017\01 January\2017.03.23_AmeriCredit Automob...,2017\03 March\2017.05.02_AmeriCredit Automobil...,2017\06 June\2017.07.01_AmeriCredit Automobile...,2017\05 May\2017.06.15_Drive Auto Receivables ...,2017\06 June\2017.07.20_Drive Auto Receivables...,2017\09 September\2017.10.11_Drive Auto Receiv...,2017\01 January\2017.02.15_Santander Drive Aut...,2017\04 April\2017.05.18_Santander Drive Auto ...,2017\08 August\2017.09.07_Santander Drive Auto...
1,2017\02 February\2017.03.23_AmeriCredit Automo...,2017\04 April\2017.06.23_AmeriCredit Automobil...,2017\07 July\2017.08.25_AmeriCredit Automobile...,2017\06 June\2017.07.17_Drive Auto Receivables...,2017\07 July\2017.08.15_Drive Auto Receivables...,2017\10 October\2017.11.15_Drive Auto Receivab...,2017\02 February\2017.03.15_Santander Drive Au...,2017\05 May\2017.06.15_Santander Drive Auto Re...,2017\09 September\2017.10.16_Santander Drive A...
2,2017\03 March\2017.04.24_AmeriCredit Automobil...,2017\05 May\2017.07.24_AmeriCredit Automobile ...,2017\08 August\2017.09.25_AmeriCredit Automobi...,2017\07 July\2017.08.15_Drive Auto Receivables...,2017\08 August\2017.09.15_Drive Auto Receivabl...,None,2017\03 March\2017.04.17_Santander Drive Auto ...,2017\06 June\2017.07.17_Santander Drive Auto R...,None
3,2017\04 April\2017.05.23_AmeriCredit Automobil...,2017\06 June\2017.08.23_AmeriCredit Automobile...,None,2017\08 August\2017.09.15_Drive Auto Receivabl...,2017\09 September\2017.10.16_Drive Auto Receiv...,None,2017\04 April\2017.05.15_Santander Drive Auto ...,2017\07 July\2017.08.15_Santander Drive Auto R...,None
4,2017\05 May\2017.06.23_AmeriCredit Automobile ...,2017\08 August\2017.09.25_AmeriCredit Automobi...,None,2017\09 September\2017.10.16_Drive Auto Receiv...,2017\10 October\2017.11.15_Drive Auto Receivab...,None,2017\05 May\2017.06.15_Santander Drive Auto Re...,2017\08 August\2017.09.15_Santander Drive Auto...,None


In [6]:
deal = 'DRIVE 2017-1'
path = 'U:\\AutoLoanTapes\\Drive Auto Receivables Trust\\DRIVE 2017-1\\' + dfTapes[deal][0]
print(path)
df = pd.read_csv(path, memory_map=True, low_memory=False)

summ['A1'].value = deal +'  '+'Month: 1'
summ['J1'].value = deal +'  '+'Month: 1'
summ.range('Summary.Statistics').clear_contents()

U:\AutoLoanTapes\Drive Auto Receivables Trust\DRIVE 2017-1\2017\05 May\2017.06.15_Drive Auto Receivables Trust 2017-1 Data Tape.csv


In [14]:
histogram(df)
obligorMetrics = ['obligorCreditScore', 'obligorIncomeVerificationLevelCode', 'obligorEmploymentVerificationCode','paymentToIncomePercentage']
creditMetrics = ['originalLoanTerm', 'remainingTermToMaturityNumber','originalInterestRatePercentage','LTV','subvented']
vehicleMetrics = ['vehicleNewUsedCode', 'vehicleManufacturerName','vehicleModelName', 'vehicleModelYear','vehicleTypeCode',
                  'obligorGeographicLocation']
vehicleStats(vehicleMetrics, df, groupDict)
creditStats(creditMetrics,df,groupDict)
obligorStats(obligorMetrics,df,groupDict)
stratTable(df, stratDict)

['paymentToIncomePercentage', 'obligorCreditScore', 'LTV', 'originalInterestRatePercentage']


C:\ProgramData\Anaconda2\lib\site-packages\numpy\lib\function_base.py:747: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= mn)
C:\ProgramData\Anaconda2\lib\site-packages\numpy\lib\function_base.py:748: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= mx)


       pctDeal
new   0.240876
used  0.759124
            pctDeal
NISSAN     0.137297
TOYOTA     0.084443
CHEVROLET  0.079362
6
           pctDeal
Altima    0.042619
Ram 1500  0.030562
Sentra    0.029728
10
       pctDeal
2017  0.189518
2014  0.177302
2016  0.154054
14
                          0
Model Year Range  2002-2018
    pctDeal
1  0.599776
3  0.274409
2  0.125816
20
     pctDeal
TX  0.157021
FL  0.111069
GA  0.074982
24
                  pctDeal
Not Subvented    0.974983
Rate Subvention  0.024131
Cash Subvention  0.000037
('No Score %:', 0.1499082295748797)
                                     pctDeal
obligorIncomeVerificationLevelCode          
Stated, Not Verified                0.841028
Stated, Verified                    0.158972
                                    pctDeal
obligorEmploymentVerificationCode          
Stated, Not Verified               0.814000
Stated, Third Party Verified       0.132861
Not Stated, Not Verified           0.053138
['Fico Score', 'APR', 'LTV', 

             assetNumber reportingPeriodActualEndBalanceAmount                 pctDeal originalLoanTerm remainingTermToMaturityNumber paymentToIncomePercentage originalInterestRatePercentage currentDelinquencyStatus
                   count                                   sum          mean       sum         <lambda>                      <lambda>                  <lambda>                       <lambda>                     mean
PTI                                                                                                                                                                                                                    
[-1.0, 0.02)         454                          7.686506e+06  16930.629427  0.005559        68.981128                     68.340501                  0.013176                       0.130654                 0.599119
[0.02, 0.04)        2055                          3.069464e+07  14936.563474  0.022200        67.890206                     67.691656   

In [8]:
def histogram(data):
    binDict = {
        'obligorCreditScore':{
            'bin':100,
            'name': 'Fico Score Histogram',
            'xLabel':'Credit Score',
            'yLabel':'Current Balance',
            'title':'Credit Score Distribution'
        },
        'originalInterestRatePercentage':{
            'bin':75,
            'name':'APR Histogram',
            'xLabel':'Original APR (%)',
            'yLabel':'Current Balance',
            'title':'APR Distribution'
        },
        'LTV':{
            'bin':100,
            'name':'LTV Histogram',
            'xLabel':'Loan to Value (%)',
            'yLabel':'Current Balance',
            'title':'LTV Distribution'
        },
        'paymentToIncomePercentage':{
            'bin':100,
            'name':'PTI Histogram',
            'xLabel':'Payment to Income (%)',
            'yLabel':'Current Balance',
            'title':'PTI Distribution'
        }
    }
    
    data['pctDeal'] = (data['reportingPeriodActualEndBalanceAmount']/(data['reportingPeriodActualEndBalanceAmount'].sum()))
    data['LTV'] = (data['originalLoanAmount']/data['vehicleValueAmount'])
    lis = list(binDict.keys())
    print(lis)
    for i in range(0,len(lis)):
        data[lis[i]] = pd.to_numeric(data[lis[i]], errors='coerce')
        histrange = [data[lis[i]].min(), data[lis[i]].max()]
        fig, ax = plt.subplots(figsize=(4,2.3))
        n, bins, patches = ax.hist(data[lis[i]], binDict[lis[i]]['bin'], histrange, weights=data['pctDeal'])
        ax.set_xlabel(binDict[lis[i]]['xLabel'])
        ax.set_ylabel(binDict[lis[i]]['yLabel'])
        ax.set_title(binDict[lis[i]]['title'])
        ax.get_yaxis().set_major_formatter(FuncFormatter(lambda x, _: '{:.0%}'.format(x)))
        if lis[i] != 'obligorCreditScore':
            ax.get_xaxis().set_major_formatter(FuncFormatter(lambda x, _: '{:.0%}'.format(x)))
        
        summ.pictures.add(fig, name=binDict[lis[i]]['name'], update=True)

In [9]:
groupDict = {
    'vehicleNewUsedCode':{
        1:'new',
        2:'used'
    },
    'vehicleTypeCode':{
        1:'Car',
        2:'Truck',
        3:'SUV',
        4:'Motorcycle',
        98:'Other',
        99:'Unknown'
    },
    'subvented':{
        0:'Not Subvented',
        1:'Rate Subvention',
        2:'Cash Subvention',
        3:'Rate & Cash Subvention',
        98:'Other Subvention'
    },
    'obligorIncomeVerificationLevelCode':{
    1:'Not Stated, Not Verified',
    2:'Stated, Not Verified',
    3:'Stated, Verified',
    4:'Stated, Verified: 12 M',
    5:'Stated, Verified: 24 M'
    },
    'obligorEmploymentVerificationCode':{
    1:'Not Stated, Not Verified',
    2:'Stated, Not Verified',
    3:'Stated, Third Party Verified'
    }
    #'vehicleValueAmount': 
}

In [10]:
def vehicleStats(metrics, data, mappingDict):
    WA = lambda x: np.average(x, axis=0, weights = data.loc[x.index, 'reportingPeriodActualEndBalanceAmount'])
    data['pctDeal'] = (data['reportingPeriodActualEndBalanceAmount']/(data['reportingPeriodActualEndBalanceAmount'].sum()))
    data['LTV'] = (data['originalLoanAmount']/data['vehicleValueAmount']) #Loan to value ratio
    row = 3
    col = 'A'
    dfVehicleStats = pd.DataFrame()
    for i in range(0, len(metrics)):
        if metrics[i] in ['vehicleNewUsedCode', 'vehicleModelYear', 'vehicleTypeCode', 'vehicleValueAmount']:
            data[metrics[i]] = pd.to_numeric(data[metrics[i]], errors='coerce')
        if metrics[i] in['vehicleManufacturerName', 'vehicleModelName', 'obligorGeographicLocation', 'vehicleModelYear', 'vehicleTypeCode']:
            if metrics[i] == 'vehicleTypeCode':
                summ.range(col + str(row)).options(pd.DataFrame, header=False).value = data[['pctDeal']].groupby(data[metrics[i]]).sum().rename(index=groupDict[metrics[i]]).head(3)
            summ.range(col + str(row)).options(pd.DataFrame, header=False).value = (data[['pctDeal']].groupby(data[metrics[i]]).sum()).sort_values(['pctDeal'], ascending=False).head(3)
            dfNew = (data[['pctDeal']].groupby(data[metrics[i]]).sum()).sort_values(['pctDeal'], ascending=False).head(3)
            del dfNew.index.name
            print(dfNew)
            print(row)
            row = row + 4
            if metrics[i] == 'vehicleModelYear':
                yearRange = [str(data[metrics[i]].min()) + '-' + str(data[metrics[i]].max())]
                summ.range(col + str(row)).options(pd.DataFrame, header=False).value = pd.DataFrame(yearRange, index=['Model Year Range'])
                dfYearRange = pd.DataFrame(yearRange, index=['Model Year Range'])
                print(dfYearRange)
                row = row + len(dfYearRange.index) + 1
            continue
        else:
            summ.range(col + str(row)).options(pd.DataFrame, header=False).value = data[['pctDeal']].groupby(data[metrics[i]]).sum().rename(index=groupDict[metrics[i]]).head(3)
            stats = data[['pctDeal']].groupby(data[metrics[i]]).sum().rename(index=groupDict[metrics[i]])
            del stats.index.name
            print(stats)
            row = row + len(stats.index) + 1
            
def creditStats(metrics, data, mappingDict):
    row = 41
    col = 'A'
    WA = lambda x: np.average(x, axis=0, weights = data.loc[x.index, 'reportingPeriodActualEndBalanceAmount'])
    data['pctDeal'] = (data['reportingPeriodActualEndBalanceAmount']/(data['reportingPeriodActualEndBalanceAmount'].sum()))
    data['LTV'] = (data['originalLoanAmount']/data['vehicleValueAmount']) #Loan to value ratio
    creditLis = []
    for i in range(0, len(metrics)):
        data[metrics[i]] = pd.to_numeric(data[metrics[i]], errors='coerce')
        if metrics[i] == 'subvented':
            data[metrics[i]] = data[metrics[i]].fillna(3)
            summ.range(col + str(row)).options(pd.DataFrame, header=False).value = data[['pctDeal']].groupby(data[metrics[i]]).sum().rename(index=groupDict[metrics[i]]).head(3)
            stats = data[['pctDeal']].groupby(data[metrics[i]]).sum().rename(index=groupDict[metrics[i]]).head(3)
            row = row + len(stats.index) + 1
            del stats.index.name
            print(stats)
            continue
        creditLis.append([WA(data[metrics[i]])])
    summ.range('B' + str(row)).value = creditLis
    row = row + len(creditLis) + 1
    return creditLis #original term, remaining term, original interenst rate, LTV

def obligorStats(metrics, data, mappingDict):
    WA = lambda x: np.average(x, axis=0, weights = data.loc[x.index, 'reportingPeriodActualEndBalanceAmount'])
    data['pctDeal'] = (data['reportingPeriodActualEndBalanceAmount']/(data['reportingPeriodActualEndBalanceAmount'].sum()))
    data['LTV'] = (data['originalLoanAmount']/data['vehicleValueAmount']) #Loan to value ratio
    row = 29
    col = 'A'
    obligorLis = []
    for i in range(0,len(metrics)):
        data[metrics[i]] = pd.to_numeric(data[metrics[i]], errors='coerce')
        if metrics[i] == 'obligorCreditScore':
            noScore = float(data[metrics[i]].isnull().sum())/len(data.index)
            print('No Score %:', noScore)
            obligorLis.append([noScore])
            newData = data[['obligorCreditScore', 'pctDeal']].dropna()
            obligorLis.append([WA(newData[metrics[i]])])
            continue
        if metrics[i] in['obligorIncomeVerificationLevelCode', 'obligorEmploymentVerificationCode']:
            summ.range(col + str(row)).options(pd.DataFrame, header=False).value = data[['pctDeal']].groupby(data[metrics[i]]).sum().rename(index=groupDict[metrics[i]]).sort_values(['pctDeal'], ascending=False).head(3)
            dfNew = data[['pctDeal']].groupby(data[metrics[i]]).sum().rename(index=groupDict[metrics[i]]).sort_values(['pctDeal'], ascending=False).head(3)
            row = row + 3 + 1
            print(dfNew)
            continue
        obligorLis.append([WA(data[metrics[i]])]) #1:percentage of no score, 2: WA Credit Score, 3: WA PTI
    summ.range('B' + str(row)).value = obligorLis
    row = row + len(obligorLis) + 1
    return obligorLis

In [11]:
stratDict = {
    'Fico Score':{
        'bin':[-1, 440, 500, 560, 620, 680, 740, 800, 1000],
        'label':['[-1, 440)', '[440, 500)', '[500, 560)', '[560, 620)', '[620, 680)', '[680, 740)', '[740, 800)', '[800, 1000)'],
        'index':['0 - 440', '441 - 500', '501 - 560', '561 - 620', '621 - 680', '681 - 740', '741 - 800', '801 +'],
        'histBin':100
    },
    'APR':{
        'bin':[0, .04, .08, .12, .16, .20, .24, .28, 1000],
        'label':['[0.0, 0.04)', '[0.04, 0.08)', '[0.08, 0.12)', '[0.12, 0.16)', '[0.16, 0.20)', '[0.20, 0.24)', '[0.24, 0.28)', '[0.28, 1000)'],
        'index': ['0.00% - 4.00%', '4.01% - 8.00%', '8.01% - 12.00%', '12.01% - 16.00%','16.01% - 20.00%', '20.01% - 24.00%', '24.01% - 28.00%', '28.00% +'],
        'histBin':75
    },
    'LTV':{
        'bin':[-1, .7, .8, .9, 1, 1.1, 1.2, 1.3, 1.4, 1000],
        'label':['[-1.0, 0.7)', '[0.7, 0.8)', '[0.8, 0.9)', '[0.9, 1.0)', '[1.0, 1.1)','[1.1, 1.2)', '[1.2, 1.3)', '[1.3, 1.4)', '[1.4, 1000)'],
        'index':['0.00% - 70.00%', '70.01% - 80.00%', '80.01% - 90.00%', '90.01% - 100.00%', '100.01% - 110.00%','110.01% - 120.00%', '120.01% - 130.00%', '130.01% - 140.00%', '140.01% +'],
        'histBin':100
        #['[-1.0, 0.7)', '[0.7, 0.8)', '[0.8, 0.9)', '[0.9, 1.0)', '[1.0, 1.1)','[1.1, 1.2)', '[1.2, 1.3)', '[1.3, 1.4)', '[1.4, 1000.0)']
    },
    'PTI':{
        'bin':[-1, .02, .04, .06, .08, .1, .12, .14, .16, 1000],
        'label':['[-1.0, 0.02)', '[0.02, 0.04)', '[0.04, 0.06)', '[0.06, 0.08)','[0.08, 0.1)', '[0.1, 0.12)', '[0.12, 0.14)', '[0.14, 0.16)', '[0.16, 1000)'],
        'index':['0.00% - 2.00%', '2.01% - 4.00%', '4.01% - 6.00%', '6.01% - 8.00%','8.01% - 10.00%', '10.01% - 12.00%', '12.01% - 14.00%','14.01% - 16.00%', '16.01% +'],
        'histBib':100
#['[-1.0, 0.02)', '[0.02, 0.04)', '[0.04, 0.06)', '[0.06, 0.08)','[0.08, 0.1)', '[0.1, 0.12)', '[0.12, 0.14)', '[0.14, 0.16)', '[0.16, 1000.0)']
    }
}

In [13]:
def stratTable(data, stratDict):
    data  = data[data.zeroBalanceCode.isin(['-'])]
    WA = lambda x: np.average(x, axis=0, weights = data.loc[x.index, 'reportingPeriodActualEndBalanceAmount'])
    data['pctDeal'] = (data['reportingPeriodActualEndBalanceAmount']/(data['reportingPeriodActualEndBalanceAmount'].sum()))
    data['LTV'] = (data['originalLoanAmount']/data['vehicleValueAmount']) #Loan to value ratio
    
    a = OrderedDict([
    ('assetNumber',['count']),
    ('reportingPeriodActualEndBalanceAmount', ['sum','mean']),
    ('pctDeal', ['sum']),
    ('originalLoanTerm',[WA]),
    ('remainingTermToMaturityNumber',[WA]),
    ('paymentToIncomePercentage',[WA]),
    #('LTV',[WA]),
    ('originalInterestRatePercentage',[WA]),
    ('currentDelinquencyStatus',['mean'])
    ])
    
    keyList = ['Fico Score', 'APR', 'LTV', 'PTI']
    print(keyList)
    colsKeyList = list(a.keys())
    stratMetrics = ['obligorCreditScore', 'originalInterestRatePercentage', 'LTV', 'paymentToIncomePercentage']
    row = 4
    row2 = 4
    for i in range(0, len(stratMetrics)):
        data[stratMetrics[i]] = pd.to_numeric(data[stratMetrics[i]], errors='coerce')
        data[keyList[i]] = pd.cut(data[stratMetrics[i]], stratDict[keyList[i]]['bin'], right=True, labels=stratDict[keyList[i]]['label'])
        if stratMetrics[i] == 'obligorCreditScore':
            data[keyList[i]] = data[keyList[i]].cat.add_categories('No Score').fillna('No Score')
        summ.range('j' + str(row)).options(pd.DataFrame, header=False).value = data[colsKeyList].groupby(by=[data[keyList[i]]]).agg(a)
        stratTable = data[colsKeyList].groupby(by=[data[keyList[i]]]).agg(a)
        
        #stratTable = stratTable.reindex(keyList[i]['label'])
        print(stratTable)
        row = row + len(stratTable.index) + 2